In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine # database connection
import sqlite3
from pandas.io.json import json_normalize
import requests

# Create database

This step should only be execute the first time. After the database has been created the first time, then creating a database is deprecated. If we create the database again we will acidentially lose of the the prior data we have collected.

In [ ]:
# create a database that will hold all ticket touting information
# e.g. table for artist_ids and information, 
# table for concerts affiliated for a particular genre, etc.

# Initializes database with filename 311_8M.db in current directory

### THIS IS THE LINE TO RUN TO CREATE THE DATABASE:
###disk_engine = create_engine('sqlite:///ticket_touting.db')


# Create a function!

This function collects all artist information based on their genre. It then creates a column to specifiy wich genre they are, and then saves all artists into a single database.

In [2]:
def get_artist_by_genre(genre, per_page=5000):
    
    """Collect all artists the your genre of choice and save dataframe
    Then use data frame to store into database as a table

    Call URL for all electronic artists
    per_page = 5000 because it is the max"""
    
    # The genre must be lowercase so let's account for that
    genre = str.lower(genre)

    
    # string together url to call api
    # client_id is the api key
    # it then mashes together the api key with the desired genre and the number of results you want
    
    client_id='NDQxMDA3NXwxNDU5MDI4MzEx'
    get_artists_url = ('https://api.seatgeek.com/2/performers?format=json&client_id='+
                       client_id+'&genres.name='+genre+'&per_page='+str(per_page))
        
    
    # use url structure to call api key
    # then return json results into dataframe.
    # I am normalizing the 'performers' section of the json
    # because the other section is just meta data info. 
    # i.e. how many artists there are and other small info that 
    # can be derived from the infomatino in performers.
    
    r = requests.get(get_artists_url)
    json_all_artists = r.json()
    df_all_artists = json_normalize(json_all_artists)
    try: 
        all_artists_ids = json_normalize(df_all_artists['performers'][0])


        # once the json have been converted to a dataframe
        # I drop a couple of features: genres and taxonomies.
        # Both of the columns cause problems when we try to store them
        # into a SQL database. Both of the columns are not needed.
        # We know the genre already, so we will make a column to replace to droped 'genres.'
        # The taxonomies data does not seem to be useful. Almost all of them return the same thing.


        all_artists_ids = all_artists_ids.drop(['genres', 'taxonomies'], 1)
        # SeetGeek has a problem generating 'hip-hop' but it works
        # if you ping the api with hip or hop. Therefore, if either of those
        # are used then well save the genre column accordingly.
        if genre == 'hip' or genre == 'hop':
            all_artists_ids['genres'] = 'hip-hop'
        else:
            all_artists_ids['genres'] = genre

        # All of the columns that are strings are converted to them.
        # When the raw data is returned from the API it is not coming back
        # as a proper string, and therefore cannot be put into a SQL database.

        all_artists_ids[['divisions', 'has_upcoming_events', 'home_venue_id', 'id', 'image', 'images.huge',
                         'links', 'name', 'score', 'short_name', 'slug', 'stats.event_count', 
                         'type', 'url']] = all_artists_ids[['divisions', 'has_upcoming_events', 'home_venue_id',
                                                            'id', 'image', 'images.huge', 'links', 'name', 
                                                            'score', 'short_name', 'slug', 'stats.event_count',
                                                            'type', 'url']].astype('str')

        # Set the is as the index colum. All ids are (or should be) unique
        all_artists_ids = all_artists_ids.set_index('id')

        # Append the data collected to the artist database
        all_artists_ids.to_sql('all_artist_data', 'sqlite:///ticket_touting.db', if_exists='append', index=True, index_label='id')

        # Now That the data has been appended lets de-duplicate our table
        # To do this I am going to conncect to the data base:
        con = sqlite3.connect('ticket_touting.db')
        cur = con.cursor()

        # Then query the the artist table, only selecting distinc/unique instances
        query = "SELECT DISTINCT * FROM all_artist_data"
        cur.execute(query)

        # From there I will execute the query and save it as a dataframe with the
        # the appropriate columns
        column_names = ['id', 'divisions', 'has_upcoming_events', 'home_venue_id', 'image', 'images.huge', 'links', 'name',
                        'score', 'short_name', 'slug', 'stats.event_count', 'type', 'url', 'genres']

        # Once the 'fect' is executed with the correct columns I am going
        # to set the 'id' column as the index, and then push all of the
        # distinct/unique artist to the 'all_artists_data' table within
        # the 'ticket_touting' database

        pd.DataFrame(cur.fetchall(),
                     columns=column_names).set_index('id').to_sql('all_artist_data',
                                                                  'sqlite:///ticket_touting.db',
                                                                  if_exists='replace', index=True, index_label='id')
        return 'Finished adding {} artists!'.format(genre)
    
    except: 
        print '''This is an incorrect genre.
       \nYou should make sure that your genre does not have any spaces or special characters.'''


# Test

Let's get data for just the electronic artist and see a snapshot of the stored data.

In [3]:
get_artist_by_genre('electronic')

'Finished adding electronic artists!'

In [3]:
# Now That the data has been appended lets de-duplicate our table
# To do this I am going to conncect to the data base:

con = sqlite3.connect('ticket_touting.db')
cur = con.cursor()

# Then query the the artist table, only selecting distinc/unique instances
query_test = "SELECT * FROM all_artist_data LIMIT 3"
cur.execute(query_test)

# From there I will execute the query and save it as a dataframe with the
# the appropriate columns
column_names = ['id', 'divisions', 'has_upcoming_events', 'home_venue_id', 'image', 'images.huge', 'links', 'name',
                    'score', 'short_name', 'slug', 'stats.event_count', 'type', 'url', 'genres']

# Once the 'fect' is executed with the correct columns I am going
# to set the 'id' column as the index.

pd.DataFrame(cur.fetchall(),
             columns=column_names).set_index('id')

,divisions,has_upcoming_events,home_venue_id,image,images.huge,links,name,score,short_name,slug,stats.event_count,type,url,genres
id,,,,,,,,,,,,,,
7960,nan,False,nan,https://chairnerd.global.ssl.fastly.net/images...,https://chairnerd.global.ssl.fastly.net/images...,[],Swedish House Mafia,0.734832,Swedish House Mafia,swedish-house-mafia,0,band,https://seatgeek.com/swedish-house-mafia-tickets,electronic
38619,nan,False,nan,nan,nan,[],Sinden,0.691532,Sinden,sinden,0,band,https://seatgeek.com/sinden-tickets,electronic
25737,nan,False,nan,nan,nan,[],Trent Cantrelle,0.691532,Trent Cantrelle,trent-cantrelle,0,band,https://seatgeek.com/trent-cantrelle-tickets,electronic


# Voilà! Let's get everything

- Now that it worked lets collect all artist information and store it into a SQL database. 
- We will be able to use that data later to collect other data.
    - I've noticed that SeatGeek's API has trouble return results if the genre has a space or special character in it. For that reason Rock will all be stored as one genre, and 'Hip' is short for hip-hop.




In [5]:
all_seetgeek_genres = ['Alternative', 'Rock', 'Country', 'Electronic', 'Hip',
                       'Indie', 'Pop', 'Rnb', 'Alternative', 'Blues', 'Classical',
                       'Folk', 'Jazz', 'Latin', 'Punk', 'Rap', 'Reggae', 'Soul']

In [6]:
for i in range(pd.DataFrame(all_seetgeek_genres).shape[0]):
    print all_seetgeek_genres[i]
    get_artist_by_genre(all_seetgeek_genres[i])

Alternative
Rock
Country
Electronic
Hip
Indie
Pop
Rnb
Alternative
Blues
Classical
Folk
Jazz
Latin
Punk
Rap
Reggae
Soul


# Next up, more data!
Now that I have collected all artist information I'm going to use that in order to collected data for all concerts for earch artist.

**I will use the genre names from the SQL artist table in order to search seetgeek's api.**

In [2]:
con = sqlite3.connect('ticket_touting.db')
cur = con.cursor()
query_test = "SELECT DISTINCT genres FROM all_artist_data"
cur.execute(query_test)
all_seetgeek_genres = pd.DataFrame(cur.fetchall())
all_seetgeek_genres

,0
0,electronic
1,hip-hop
2,alternative
3,rock
4,country
5,indie
6,pop
7,rnb
8,blues
9,classical


**I'm using the distinct genres to search through the SQL table. The way that I will search and get results is shown in the code below. This code will be used in a function.**

In [80]:
query_test = "SELECT * FROM all_artist_data WHERE genres = '{}'".format(distinct_genres['genres'][0])
cur.execute(query_test)

# From there I will execute the query and save it as a dataframe with the
# the appropriate columns
column_names = ['id', 'divisions', 'has_upcoming_events', 'home_venue_id', 'image', 'images.huge', 'links', 'name',
                    'score', 'short_name', 'slug', 'stats.event_count', 'type', 'url', 'genres']

# Once the 'fect' is executed with the correct columns I am going
# to set the 'id' column as the index.

x = pd.DataFrame(cur.fetchall(),
             columns=column_names).set_index('id')

In [81]:
x

,divisions,has_upcoming_events,home_venue_id,image,images.huge,links,name,score,short_name,slug,stats.event_count,type,url,genres
id,,,,,,,,,,,,,,
7960,nan,False,nan,https://chairnerd.global.ssl.fastly.net/images...,https://chairnerd.global.ssl.fastly.net/images...,[],Swedish House Mafia,0.734832,Swedish House Mafia,swedish-house-mafia,0,band,https://seatgeek.com/swedish-house-mafia-tickets,electronic
38619,nan,False,nan,nan,nan,[],Sinden,0.691532,Sinden,sinden,0,band,https://seatgeek.com/sinden-tickets,electronic
25737,nan,False,nan,nan,nan,[],Trent Cantrelle,0.691532,Trent Cantrelle,trent-cantrelle,0,band,https://seatgeek.com/trent-cantrelle-tickets,electronic
18965,nan,True,nan,nan,nan,[],Nina Kraviz,0.68924,Nina Kraviz,nina-kraviz,1,band,https://seatgeek.com/nina-kraviz-tickets,electronic
58199,nan,False,nan,https://chairnerd.global.ssl.fastly.net/images...,https://chairnerd.global.ssl.fastly.net/images...,[],Lady Starlight,0.687344,Lady Starlight,lady-starlight,0,band,https://seatgeek.com/lady-starlight-tickets,electronic
22817,nan,False,nan,nan,nan,[],Huoratron,0.678964,Huoratron,huoratron,0,band,https://seatgeek.com/huoratron-tickets,electronic
33619,nan,True,nan,nan,nan,[],Laurent Garnier,0.678358,Laurent Garnier,laurent-garnier,1,band,https://seatgeek.com/laurent-garnier-tickets,electronic
17603,nan,True,nan,https://chairnerd.global.ssl.fastly.net/images...,https://chairnerd.global.ssl.fastly.net/images...,[],Sensation,0.665095,Sensation,sensation,4,band,https://seatgeek.com/sensation-tickets,electronic
16783,nan,True,nan,nan,nan,[],Lee Burridge,0.661718,Lee Burridge,lee-burridge,4,band,https://seatgeek.com/lee-burridge-tickets,electronic


**I'm making a function to do this data collection**

In [83]:
x.shape[0]

1825

In [40]:
def get_concert_by_artist(genre_in_database):
    con = sqlite3.connect('ticket_touting.db')
    cur = con.cursor()
    query = "SELECT id, genres, short_name FROM all_artist_data WHERE genres = '{}'".format(genre_in_database)
    cur.execute(query)
    x = pd.DataFrame(cur.fetchall(),columns=['id', 'genres', 'short_name'])
    
    master_concerts = pd.DataFrame()
    
    for i in range(x.shape[0]):
        try:    
            artist_concert_info = "https://api.seatgeek.com/2/events?performers.id={}&client_id=NDQxMDA3NXwxNDU5MDI4MzEx&per_page=5000&datetime_utc.gt=2012-01-01".format(
                x['id'][i])

            r = requests.get(artist_concert_info)
            json_concert_per_artist = r.json()
            y = json_normalize(json_concert_per_artist)
            if y['events'][0] != []:
                single_artist_df = json_normalize(y['events'][0])
                single_artist_df['artist_name'] = x['short_name'][i]
                single_artist_df['artist_id'] = x['id'][i]
                single_artist_df['artist_genre'] = x['genres'][i]
                master_concerts = master_concerts.append(single_artist_df)
                


        except:
            'placeholder'
            
        if i % 50 == 0:
            print 'Searching #{} out of {}'.format(i, x.shape[0])
                

    # We will only put the data into a SQL database 
    # once all of the data has been collected.
    # Once the data is added to the database then we will run a 
    # distinct SQL function in order to de-duplicate the data

    print 'Making the database for {}'.format(genre_in_database)

    master_concerts = master_concerts.drop(['links', 'performers',
                                            'taxonomies', 'venue.links',
                                            'venue.url'], 1).set_index('id')
    master_concerts.to_sql('all_concert_data', 'sqlite:///ticket_touting.db',
                       if_exists='append', index=True, index_label='id')

    print 'Making the de-duplicated database!'

    # Now That the data has been appended lets de-duplicate our table
    # To do this I am going to conncect to the data base:
    con = sqlite3.connect('ticket_touting.db')
    cur = con.cursor()

    # Then query the the artist table, only selecting distinc/unique instances
    query = "SELECT DISTINCT * FROM all_concert_data"
    cur.execute(query)

    # From there I will execute the query and save it as a dataframe with the
    # the appropriate columns
    column_names = ['id', 'announce_date', 'created_at', 'date_tbd', 'datetime_local', 'datetime_tbd',
                    'datetime_utc', 'general_admission', 'score','short_title','stats.average_price',
                     'stats.highest_price','stats.listing_count','stats.lowest_price', 
                    'stats.lowest_price_good_deals','time_tbd','title','type','url','venue.address',
                    'venue.city','venue.country','venue.display_location','venue.extended_address',
                    'venue.id','venue.location.lat','venue.location.lon','venue.name',
                    'venue.postal_code','venue.score','venue.slug', 'venue.state', 
                    'venue.timezone','visible_until_utc','artist_name','artist_id','artist_genre']



    # Once the 'fect' is executed with the correct columns I am going
    # to set the 'id' column as the index, and then push all of the
    # distinct/unique artist to the 'all_artists_data' table within
    # the 'ticket_touting' database

    pd.DataFrame(cur.fetchall(),
                 columns=column_names).set_index('id').to_sql('all_concert_data',
                                                              'sqlite:///ticket_touting.db',
                                                              if_exists='replace', index=True, index_label='id')

    return 'Done!'

In [41]:
get_concert_by_artist('electronic')

Searching #0 out of 1825
Searching #50 out of 1825
Searching #100 out of 1825
Searching #150 out of 1825
Searching #200 out of 1825
Searching #250 out of 1825
Searching #300 out of 1825
Searching #350 out of 1825
Searching #400 out of 1825
Searching #450 out of 1825
Searching #500 out of 1825
Searching #550 out of 1825
Searching #600 out of 1825
Searching #650 out of 1825
Searching #700 out of 1825
Searching #750 out of 1825
Searching #800 out of 1825
Searching #850 out of 1825
Searching #900 out of 1825
Searching #950 out of 1825
Searching #1000 out of 1825
Searching #1050 out of 1825
Searching #1100 out of 1825
Searching #1150 out of 1825
Searching #1200 out of 1825
Searching #1250 out of 1825
Searching #1300 out of 1825
Searching #1350 out of 1825
Searching #1400 out of 1825
Searching #1450 out of 1825
Searching #1500 out of 1825
Searching #1550 out of 1825
Searching #1600 out of 1825
Searching #1650 out of 1825
Searching #1700 out of 1825
Searching #1750 out of 1825
Searching #1800

'Done!'

Let's do a query and make sure the data stored properly

In [42]:
con = sqlite3.connect('ticket_touting.db')
cur = con.cursor()
query = "SELECT * FROM all_concert_data"
cur.execute(query)

column_names = ['id', 'announce_date', 'created_at', 'date_tbd', 'datetime_local', 'datetime_tbd',
                'datetime_utc', 'general_admission', 'score','short_title','stats.average_price',
                 'stats.highest_price','stats.listing_count','stats.lowest_price', 
                'stats.lowest_price_good_deals','time_tbd','title','type','url','venue.address',
                'venue.city','venue.country','venue.display_location','venue.extended_address',
                'venue.id','venue.location.lat','venue.location.lon','venue.name',
                'venue.postal_code','venue.score','venue.slug', 'venue.state', 
                'venue.timezone','visible_until_utc','artist_name','artist_id','artist_genre']

pd.DataFrame(cur.fetchall(), columns=column_names).set_index('id')

,announce_date,created_at,date_tbd,datetime_local,datetime_tbd,datetime_utc,general_admission,score,short_title,stats.average_price,...,venue.name,venue.postal_code,venue.score,venue.slug,venue.state,venue.timezone,visible_until_utc,artist_name,artist_id,artist_genre
id,,,,,,,,,,,,,,,,,,,,,
3155401,2016-03-16T00:00:00,2016-03-16T00:00:00,0,2016-05-23T03:30:00,0,2016-05-23T01:30:00,NaN,0.626239,Davide Squillace - Circoloco Opening Party,None,...,DC10,07816,0.000000,dc10,07,Europe/Madrid,2016-05-23T22:00:00,Damian Lazarus,23393,electronic
3265144,2016-05-01T00:00:00,2016-05-01T00:00:00,0,2016-06-17T03:30:00,0,2016-06-17T01:30:00,NaN,NaN,Nic Fanciulli with Damian Lazarus and Yousef,None,...,Razzmatazz Clubs,08018,0.000000,razzmatazz-clubs,None,Europe/Madrid,2016-06-17T22:00:00,Damian Lazarus,23393,electronic
3228287,2016-04-12T00:00:00,2016-04-12T00:00:00,0,2016-08-13T12:00:00,0,2016-08-13T10:00:00,NaN,0.000000,"Loveland Festival with Damian Lazarus, Fur Coat",None,...,Sloterpark,1064 GW,0.000000,sloterpark,None,Europe/Amsterdam,2016-08-13T14:00:00,Damian Lazarus,23393,electronic
3062766,2016-01-28T00:00:00,2016-01-28T00:00:00,0,2016-08-28T11:00:00,0,2016-08-28T10:00:00,1,0.615956,South West Four (SW4) Festival (Sunday Pass) w...,None,...,Clapham Common,SW4,0.659543,clapham-common,Lbh,Europe/London,2016-08-28T14:00:00,Damian Lazarus,23393,electronic
3265400,2016-05-01T00:00:00,2016-05-01T00:00:00,0,2016-05-14T15:00:00,0,2016-05-14T22:00:00,1,NaN,Sinden,None,...,Fox Theater Pomona,91766,0.617754,fox-theater-pomona,CA,America/Los_Angeles,2016-05-15T02:00:00,Sinden,38619,electronic
3225074,2016-04-11T00:00:00,2016-04-11T00:00:00,0,2016-05-29T17:00:00,0,2016-05-29T23:00:00,NaN,0.648849,Nina Kraviz - Denver,25.5,...,Beta Nightclub,80202,0.412814,beta-nightclub,CO,America/Denver,2016-05-30T03:00:00,Nina Kraviz,18965,electronic
3264577,2016-05-01T00:00:00,2016-05-01T00:00:00,0,2016-05-30T23:00:00,0,2016-05-31T03:00:00,NaN,NaN,Nina Kraviz with Bjarki,None,...,Populux,48201,0.461628,populux,MI,America/Detroit,2016-05-31T07:00:00,Nina Kraviz,18965,electronic
3138871,2016-03-07T00:00:00,2016-03-07T00:00:00,0,2016-05-07T21:00:00,0,2016-05-07T20:00:00,NaN,0.672708,Laurent Garnier,None,...,XOYO,EC2A 4AP,0.357290,xoyo,London,Europe/London,2016-05-08T00:00:00,Laurent Garnier,33619,electronic
3219118,2016-04-10T00:00:00,2016-04-10T00:00:00,0,2016-05-07T21:00:00,0,2016-05-08T01:00:00,NaN,0.658884,Sensation,None,...,New Mountain Asheville,28801,0.431002,new-mountain-asheville,NC,America/New_York,2016-05-08T05:00:00,Sensation,17603,electronic


# Collect All Data

Now I am going to collect concert data for all genres.

In [44]:
for i in range(all_seetgeek_genres.shape[0]):
    if i == 0:
        print 'skip! edm already done!'
    else:
        print all_seetgeek_genres[0][i]
        get_concert_by_artist(all_seetgeek_genres[0][i])

skip! edm already done!
hip-hop
Searching #0 out of 2246
Searching #50 out of 2246
Searching #100 out of 2246
Searching #150 out of 2246
Searching #200 out of 2246
Searching #250 out of 2246
Searching #300 out of 2246
Searching #350 out of 2246
Searching #400 out of 2246
Searching #450 out of 2246
Searching #500 out of 2246
Searching #550 out of 2246
Searching #600 out of 2246
Searching #650 out of 2246
Searching #700 out of 2246
Searching #750 out of 2246
Searching #800 out of 2246
Searching #850 out of 2246
Searching #900 out of 2246
Searching #950 out of 2246
Searching #1000 out of 2246
Searching #1050 out of 2246
Searching #1100 out of 2246
Searching #1150 out of 2246
Searching #1200 out of 2246
Searching #1250 out of 2246
Searching #1300 out of 2246
Searching #1350 out of 2246
Searching #1400 out of 2246
Searching #1450 out of 2246
Searching #1500 out of 2246
Searching #1550 out of 2246
Searching #1600 out of 2246
Searching #1650 out of 2246
Searching #1700 out of 2246
Searching #

**I'm going to execute a couple of queries to make sure that the data is stored ok. First I will do a simple query a head of the dataframe. Then I'm going to ask for all of the unique genres within the table.**

In [46]:
con = sqlite3.connect('ticket_touting.db')
cur = con.cursor()
query = "SELECT * FROM all_concert_data limit 10"
cur.execute(query)

column_names = ['id', 'announce_date', 'created_at', 'date_tbd', 'datetime_local', 'datetime_tbd',
                'datetime_utc', 'general_admission', 'score','short_title','stats.average_price',
                 'stats.highest_price','stats.listing_count','stats.lowest_price', 
                'stats.lowest_price_good_deals','time_tbd','title','type','url','venue.address',
                'venue.city','venue.country','venue.display_location','venue.extended_address',
                'venue.id','venue.location.lat','venue.location.lon','venue.name',
                'venue.postal_code','venue.score','venue.slug', 'venue.state', 
                'venue.timezone','visible_until_utc','artist_name','artist_id','artist_genre']

pd.DataFrame(cur.fetchall(), columns=column_names).set_index('id')

,announce_date,created_at,date_tbd,datetime_local,datetime_tbd,datetime_utc,general_admission,score,short_title,stats.average_price,...,venue.name,venue.postal_code,venue.score,venue.slug,venue.state,venue.timezone,visible_until_utc,artist_name,artist_id,artist_genre
id,,,,,,,,,,,,,,,,,,,,,
3155401,2016-03-16T00:00:00,2016-03-16T00:00:00,0,2016-05-23T03:30:00,0,2016-05-23T01:30:00,NaN,0.626239,Davide Squillace - Circoloco Opening Party,None,...,DC10,07816,0.000000,dc10,07,Europe/Madrid,2016-05-23T22:00:00,Damian Lazarus,23393,electronic
3265144,2016-05-01T00:00:00,2016-05-01T00:00:00,0,2016-06-17T03:30:00,0,2016-06-17T01:30:00,NaN,NaN,Nic Fanciulli with Damian Lazarus and Yousef,None,...,Razzmatazz Clubs,08018,0.000000,razzmatazz-clubs,None,Europe/Madrid,2016-06-17T22:00:00,Damian Lazarus,23393,electronic
3228287,2016-04-12T00:00:00,2016-04-12T00:00:00,0,2016-08-13T12:00:00,0,2016-08-13T10:00:00,NaN,0.000000,"Loveland Festival with Damian Lazarus, Fur Coat",None,...,Sloterpark,1064 GW,0.000000,sloterpark,None,Europe/Amsterdam,2016-08-13T14:00:00,Damian Lazarus,23393,electronic
3062766,2016-01-28T00:00:00,2016-01-28T00:00:00,0,2016-08-28T11:00:00,0,2016-08-28T10:00:00,1,0.615956,South West Four (SW4) Festival (Sunday Pass) w...,None,...,Clapham Common,SW4,0.659543,clapham-common,Lbh,Europe/London,2016-08-28T14:00:00,Damian Lazarus,23393,electronic
3265400,2016-05-01T00:00:00,2016-05-01T00:00:00,0,2016-05-14T15:00:00,0,2016-05-14T22:00:00,1,NaN,Sinden,None,...,Fox Theater Pomona,91766,0.617754,fox-theater-pomona,CA,America/Los_Angeles,2016-05-15T02:00:00,Sinden,38619,electronic
3225074,2016-04-11T00:00:00,2016-04-11T00:00:00,0,2016-05-29T17:00:00,0,2016-05-29T23:00:00,NaN,0.648849,Nina Kraviz - Denver,25.5,...,Beta Nightclub,80202,0.412814,beta-nightclub,CO,America/Denver,2016-05-30T03:00:00,Nina Kraviz,18965,electronic
3264577,2016-05-01T00:00:00,2016-05-01T00:00:00,0,2016-05-30T23:00:00,0,2016-05-31T03:00:00,NaN,NaN,Nina Kraviz with Bjarki,None,...,Populux,48201,0.461628,populux,MI,America/Detroit,2016-05-31T07:00:00,Nina Kraviz,18965,electronic
3138871,2016-03-07T00:00:00,2016-03-07T00:00:00,0,2016-05-07T21:00:00,0,2016-05-07T20:00:00,NaN,0.672708,Laurent Garnier,None,...,XOYO,EC2A 4AP,0.357290,xoyo,London,Europe/London,2016-05-08T00:00:00,Laurent Garnier,33619,electronic
3219118,2016-04-10T00:00:00,2016-04-10T00:00:00,0,2016-05-07T21:00:00,0,2016-05-08T01:00:00,NaN,0.658884,Sensation,None,...,New Mountain Asheville,28801,0.431002,new-mountain-asheville,NC,America/New_York,2016-05-08T05:00:00,Sensation,17603,electronic


In [47]:
con = sqlite3.connect('ticket_touting.db')
cur = con.cursor()
query = "SELECT DISTINCT artist_genre FROM all_concert_data"
cur.execute(query)

column_names = ['artist_genre']

pd.DataFrame(cur.fetchall(), columns=column_names)

,artist_genre
0,electronic
1,hip-hop
2,alternative
3,rock
4,country
5,indie
6,pop
7,rnb
8,blues
9,classical
